# Covid vs India 
> One of the biggest question I have been wondering is , with every adult allowed now to register, Where are vaccines slots available in india ? How can I make this decision programmitcally because trying to find information on government website is rather "sisyphus task".
So with rather self involed goal in mind, We would using publicly avaialbe data try to answer following questions
- toc: false
- branch: master
- badges: false
- comments: true
- categories: [dfs, jupyter, python]
- image: images/statistics.png
- hide: false

1. what's ground reality for Key states like Delhi, Maharastra , Tamil Nadu
2. What's ground reality for populus states like UP, MP, 
3. How does it fare around 
2. Latest vaccine slots per district and State in India
2. Top 5 States Running behind on schedule
3. Top 5 States Running ahead of everyone else.
4. When can I latest find slots in Delhi, Bombay, Chennai and Bangalore for people between 18-45



Thanks to https://github.com/bhattbhavesh91/cowin-vaccination-slot-availability/blob/main/cowin-api-availability.ipynb for doing actual work, I picked up loads of stuff from there.


In [293]:
#hide
!{sys.executable} -m pip install --user install requests
!{sys.executable} -m pip install --user install altair
!{sys.executable} -m pip install --user install pandas
!{sys.executable} -m pip install --user install geopandas

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


Get All libraries in place

In [298]:
import geopandas as gpd
import pandas as pd
import requests
import json
from collections import defaultdict
from dataclasses import dataclass, asdict, field
from datetime import datetime
from typing import List
import uuid

Lets make a data class to store our geographical and vaccination data

In [303]:
@dataclass
class District:
    district_id:int = None
    district_name:str = None
    state_id:int = None
    district_created_at: datetime = field(default_factory=datetime.now)


@dataclass
class Session:
    session_uuid:str = None
    date:datetime = None
    query_date:datetime= None
    available_capacity:int = None
    min_age_limit:int = None
    vaccine:str = None
    center_id:str = None
    district_id:str = None
    session_created_at: datetime = field(default_factory=datetime.now)
        
@dataclass
class Center:
    center_uuid:str = None
    center_id:int = None
    center_name:str = None
    state_name:str = None
    district_name:str = None
    block_name:str = None
    pincode:str = None
    lat:int = None
    lng:int = None
    from_hour:datetime = None
    to_hour:datetime = None
    fee_type:str = None
    district_id:str = None
    center_created_at: datetime = field(default_factory=datetime.now)

@dataclass
class NoSlotAvailable:
    district_id:str = None
    date:datetime = None
    no_slot_available_created_at: datetime = field(default_factory=datetime.now)

Lets call the API to get the populate geographical data

In [ ]:
MOZILLA_HEADER = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
GET_DISTRICT_DATA_API_URL = "https://cdn-api.co-vin.in/api/v2/admin/location/districts/{}"
GET_APOINTMENT_DATA_API_URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}"

DISTRICTS = defaultdict(District)
CENTERS = defaultdict(Center)
SESSIONS = defaultdict(Session)
NO_SLOT_AVAILABLE = []

MAX_NUMBER_OF_STATES = 40

for state_code in range(1, MAX_NUMBER_OF_STATES):
    headers = {'User-Agent': MOZILLA_HEADER}
    response = requests.get(GET_DISTRICT_DATA_API_URL.format(state_code), headers=headers)
    districts_data = json.loads(response.content)
    for district in districts_data['districts']:
        district_name = district['district_name']
        district_id = district['district_id']
        district = District(district_name = district_name, district_id = district_id , state_id=state_code) 
        DISTRICTS[district_id] = district

In [ ]:
DISTRICT_DF = pd.DataFrame.from_dict([asdict(district) for district in DISTRICTS.values()])

Lets now call actual api to get slots 

In [ ]:
#hide_output
MAX_DAYS = 7

#This step takes loads of time
def get_days_in_future_from_today():
    base = datetime.today()
    date_list = [base + datetime.timedelta(days=x) for x in range(MAX_DAYS)]
    return [x.strftime("%d-%m-%Y") for x in date_list]

for district_id in DISTRICTS.keys():
    print("District {}".format(district_id))
    for slot_date in get_data_fiften_days_in_future_from_today():
        URL = GET_APOINTMENT_DATA_API_URL.format(district_id, slot_date)
        response = requests.get(URL)
        if response.ok:
            resp_json = response.json()
            if resp_json["centers"]:
                for center in resp_json["centers"]:
                    center_uuid = str(uuid.uuid4())
                    center_id = center["center_id"]
                    center_name = center["name"]
                    CENTERS[center_uuid] = Center(center_uuid=center_uuid,
                                                center_id=center_id,
                                                center_name=center_name,
                                                lat=center["lat"],
                                                lng=center["long"],
                                                from_hour=center["from"],
                                                to_hour=center["to"],
                                                district_id=district_id,
                                                state_name=center["state_name"],
                                                district_name=center["district_name"],
                                                block_name=center["block_name"],
                                                pincode=center["pincode"],
                                                fee_type=center["fee_type"])
                    for session in center["sessions"]:
                        session_id = session["session_id"]
                        SESSIONS[session_id] = Session(session_uuid=session_id,
                                                       date=session["date"],
                                                       query_date=slot_date,
                                                       available_capacity=session["available_capacity"],
                                                       min_age_limit=session["min_age_limit"],
                                                       vaccine=session["vaccine"],
                                                       district_id=district_id,
                                                       center_id=center_id)
            else:
                NO_SLOT_AVAILABLE.append(NoSlotAvailable(district_id=district_id, date=slot_date))
#                 print("No slot on {} in district {}".format(slot_date, district_id))

In [ ]:
CENTER_DF = pd.DataFrame.from_dict([asdict(district) for district in CENTERS.values()])
SESSION_DF = pd.DataFrame.from_dict([asdict(session) for session in SESSIONS.values()])
NO_SLOT_AVAILABLE_DF = pd.DataFrame.from_dict([asdict(no_slot_available) for no_slot_available in NO_SLOT_AVAILABLE])

In [ ]:
NO_SLOT_AVAILABLE_DF.head()

In [ ]:
SESSION_DF.head()

In [ ]:
CENTER_DF.head()

In [ ]:
DISTRICT_DF.head()

In [ ]:
session_center_merged_df = pd.merge(SESSION_DF, CENTER_DF, on="center_id")
session_center_district_df = pd.merge(session_center_merged_df, DISTRICT_DF, left_on='district_id_x', right_on="district_id")

In [ ]:
session_center_district_df.to_csv("vaccination_slot_data.csv")
session_center_district_df

In [ ]:
# Top 5 States Running ahead of everyone else.
session_center_district_df[['date', 'state_name']].groupby('state_name').apply(lambda x : x.sort_values(by = 'date', ascending = True).head(3).reset_index(drop = True))

In [ ]:
session_center_district_df\
    [['date', 'state_name']]\
    .groupby('state_name')\
    .apply(lambda x : x.sort_values(by = 'date', ascending = True)\
    .head(1)\
    .reset_index(drop = True)).head(40)

In [ ]:
session_center_district_df\
    [session_center_district_df["min_age_limit"]<45]\
    [session_center_district_df["fee_type"]=="Free"]\
    [['date', 'state_name']]\
    .groupby('state_name')\
    .apply(lambda x : x.sort_values(by = 'date', ascending = True)\
    .head(1)\
    .reset_index(drop = True)).head(40)

In [ ]:
session_center_district_df.drop_duplicates(["date","state_id"]).groupby("state_id").apply(lambda x : x.sort_values(by = 'date', ascending = True).head(1).reset_index(drop = True)).head(40)

In [ ]:
session_center_district_df